In [3]:
from data_loader import get_dicom_paths, parse_metadata, create_labels, load_dataset
import pandas as pd

In [4]:
TRAIN_IMG_DIR = r"C:\\Users\\samya\\PyCharmProject\\Pneumonia-Detection_dataset\\data\\stage_2_train_images"
TEST_IMG_DIR  = r"C:\\Users\\samya\\PyCharmProject\\Pneumonia-Detection_dataset\\data\\stage_2_test_images"
LABEL_CSV     = r"C:\\Users\\samya\\PyCharmProject\\Pneumonia-Detection_dataset\\data\\stage_2_train_labels.csv"

In [ ]:
train_img_paths = get_dicom_paths(TRAIN_IMG_DIR, label="train")
test_img_paths  = get_dicom_paths(TEST_IMG_DIR, label="test")
labels_df       = pd.read_csv(LABEL_CSV)

Found 26684 DICOM files in the 'train' directory.
Found 3000 DICOM files in the 'test' directory.


In [8]:
# Ensure output directories exist
os.makedirs("yolo_images", exist_ok=True)
os.makedirs("yolo_labels", exist_ok=True)


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pydicom
from tqdm import tqdm

# Define paths
TRAIN_IMG_DIR = r"C:\Users\samya\PyCharmProject\Pneumonia-Detection_dataset\data\stage_2_train_images"
TEST_IMG_DIR = r"C:\Users\samya\PyCharmProject\Pneumonia-Detection_dataset\data\stage_2_test_images"
LABEL_CSV = r"C:\Users\samya\PyCharmProject\Pneumonia-Detection_dataset\data\stage_2_train_labels.csv"

# Function to convert DICOM to JPG and save it
def save_image_from_dicom(dicom_dir, output_img_dir, patient_id):
    img_path = os.path.join(output_img_dir, f"{patient_id}.jpg")
    if os.path.exists(img_path):
        return  # Skip if image already exists

    dicom_path = os.path.join(dicom_dir, f"{patient_id}.dcm")
    dicom_data = pydicom.dcmread(dicom_path)
    img_array = dicom_data.pixel_array

    # Convert to 3-channel image for YOLO compatibility
    img_rgb = np.stack([img_array] * 3, axis=-1)
    cv2.imwrite(img_path, img_rgb)

# Function to save YOLO-format label file
def save_yolo_label(label_dir, patient_id, row=None):
    img_size = 1024  # RSNA images are 1024x1024
    label_path = os.path.join(label_dir, f"{patient_id}.txt")

    with open(label_path, "a") as f:
        if row is None:
            return  # No bounding box to write

        # Extract bounding box coordinates
        x, y, w, h = row[1], row[2], row[3], row[4]

        # Convert to YOLO format: class_id cx cy width height (all normalized)
        cx = (x + w / 2) / img_size
        cy = (y + h / 2) / img_size
        nw = w / img_size
        nh = h / img_size

        f.write(f"0 {cx} {cy} {nw} {nh}\n")  # Class 0 = pneumonia

# Function to process the entire dataset and save images and labels
def prepare_yolo_dataset(dicom_dir, output_img_dir, label_dir, annotations_df):
    for row in tqdm(annotations_df.values):
        patient_id = row[0]
        target = row[5]  # 1 = pneumonia, 0 = normal

        img_path = os.path.join(output_img_dir, f"{patient_id}.jpg")

        # Skip non-pneumonia cases to save space (optional)
        if target == 0:
            continue

        # Save label and image
        save_yolo_label(label_dir, patient_id, row)
        if not os.path.exists(img_path):
            save_image_from_dicom(dicom_dir, output_img_dir, patient_id)

# Example usage
labels_df = pd.read_csv(LABEL_CSV)
prepare_yolo_dataset(TRAIN_IMG_DIR, "yolo_images", "yolo_labels", labels_df)


  0%|          | 4/30227 [00:00<00:07, 4003.15it/s]


AttributeError: module 'pydicom' has no attribute 'read_file'